In [81]:
import pandas as pd
import numpy as np
import os

import time
from alibi_detect.od import SpectralResidual

In [35]:
def get_data():
    data_path = '../../../../data/train_data/host'
    dfs = {}
    for file in os.listdir(data_path):
        print('Reading ' + file)
        dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 
    return dfs

def load_tiago_test(test):
    data_path = '../../../../data/tiago_tests/test'
    df = pd.read_csv(data_path+str(test)+'/kpis.csv')
    return df

def load_test():
    data_path = '../../../../data/test_data/host'
    df = pd.DataFrame(['item_id','name','bomc_id','timestamp','value','cmdb_id'])
    for file in os.listdir(data_path):
        df = pd.concatenate(df, pd.read_csv(data_path+'/'+file)) 
    return dfs


In [7]:
def get_past(df, curr_time, window_size):
    step_size = 1000*60
    df_new = pd.DataFrame(columns=['timestamp', 'value'])

    for time in [curr_time - step_size*x for x in range(window_size)][::-1]:
        if len(df[df.timestamp == time]) == 0:
            plug_time = np.max(df[df.timestamp < time]['timestamp'])
            row = {}
            row['timestamp'] = time
            if len(df.loc[df.timestamp==plug_time, 'value']) == 0:
                row['value'] = 0
            else: 
                row['value'] = df.loc[df.timestamp==plug_time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
        else:
            row = {}
            row['timestamp'] = time
            row['value'] = df.loc[df.timestamp==time, 'value'].values[0]
            df_new = df_new.append(row, ignore_index=True)
    return df_new

In [82]:
def get_key_thresh(host):
    df_thresh = pd.read_csv('thresh.data')
    df_thresh = df_thresh[df_thresh.host==host]

    thresh = df_thresh[['name', 'thresh']].set_index('name')
    thresh = thresh.to_dict()['thresh']
    return thresh

def find_anoms(hosts, df):
    start = time.time()
    anoms = []
    for host in hosts:
        start_host = time.time()
        print('*'*60)
        print('Checking ', host)
        thresholds = get_key_thresh(host)
        for kpi in thresholds:
            thresh = thresholds[kpi]
            print('Threshold ', thresh, kpi)
            od = SpectralResidual(
                threshold=thresh,
                window_amp=30,
                window_local=30,
                n_est_points=5,
                n_grad_points=5
            )
            df_hk = df[df.cmdb_id == host][df.name == kpi]
            
            if len(df_hk) == 0:
                print('No valid data to use, skipping')
                continue
                
            data = get_past(df_hk, np.max(df_hk.timestamp.unique()), 30)['value']
            if thresh == -1:
                if sum(data[-10:-5]) > 0:
                    print("Non-Zero Anomaly!")
                    anoms.append((host, kpi))
            elif np.isnan(thresh):
                print('NaN, Skipping!')
            else:
                outliers = od.predict(data.values)['data']
                if np.sum(outliers['is_outlier'][-10:-5]) > 0:
                    print(outliers['is_outlier'])
                    print("ST Threshold Anomaly!")
                    anoms.append((host,kpi))
        print(host, ' completed in ', time.time() - start_host)
    print('Completed detection of ', len(hosts), 'hosts in ', time.time() - start, 'seconds')
    return anoms

print(load_test(2).cmdb_id.unique())
find_anoms(load_test(2).cmdb_id.unique(), load_test(2))

['db_003' 'db_011' 'db_005' 'db_010' 'db_012' 'db_008' 'db_009' 'db_013'
 'db_006' 'db_007' 'db_004' 'db_002' 'db_001' 'container_001'
 'container_002' 'docker_002' 'docker_005' 'docker_004' 'docker_008'
 'docker_003' 'docker_001' 'docker_007' 'docker_006' 'redis_003'
 'redis_012' 'redis_007' 'redis_011' 'redis_005' 'redis_001' 'redis_008'
 'redis_009' 'redis_004' 'redis_006' 'redis_002' 'redis_010' 'os_003'
 'os_017' 'os_008' 'os_014' 'os_005' 'os_012' 'os_009' 'os_015' 'os_004'
 'os_006' 'os_020' 'os_001' 'os_011' 'os_010' 'os_007' 'os_018' 'os_013'
 'os_016' 'os_019' 'os_002' 'os_022' 'os_021']
************************************************************
Checking  db_003
Threshold  0.5792020371267232 CPU_free_pct
Threshold  0.5789452140089406 MEM_real_util
Threshold  0.5613863637942091 tnsping_result_time


<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


[0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
ST Threshold Anomaly!
Threshold  0.8002607259374916 CPU_Used_Pct
Threshold  0.5786337696298588 MEM_Used
Threshold  nan MEM_Total
NaN, Skipping!
Threshold  0.578971558855777 MEM_Used_Pct


<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


Threshold  0.5789479809646076 Used_Tbs_Size
Threshold  nan SEQ_Used_Pct
No valid data to use, skipping
Threshold  0.5753000989599808 Asm_Free_Tb
Threshold  0.5729319438826606 PGA_used_total
Threshold  0.9014230643122688 Sess_Used_Undo


...

Threshold  0.5806015196960354 FS_used_pct
Threshold  nan FS_total_space
NaN, Skipping!
Threshold  nan FS_max_util
NaN, Skipping!
Threshold  nan ICMP_ping


<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


NaN, Skipping!
Threshold  0.5933456387007685 FS_max_avail
Threshold  -1.0 Swap_used_pct
Threshold  nan Memory_total
NaN, Skipping!
os_021  completed in  3.012223958969116
Completed detection of  57  hosts in  117.10171389579773 seconds


<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]
<ipython-input-82-dcc1e2084555>:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hk = df[df.cmdb_id == host][df.name == kpi]


[('db_003', 'tnsping_result_time'),
 ('db_003', 'Session_pct'),
 ('db_003', 'DbTime'),
 ('db_003', 'Sess_Active'),
 ('db_003', 'Logic_Read_Per_Sec'),
 ('db_003', 'Exec_Per_Sec'),
 ('db_003', 'New_Tbs_Free_Gb'),
 ('db_011', 'Session_pct'),
 ('db_011', 'DbTime'),
 ('db_011', 'Sess_Active'),
 ('db_011', 'Physical_Read_Per_Sec'),
 ('db_011', 'Call_Per_Sec'),
 ('db_011', 'Login_Per_Sec'),
 ('db_005', 'CPU_Used_Pct'),
 ('db_005', 'SeqRead_Per_Sec'),
 ('db_005', 'Session_pct'),
 ('db_005', 'LFParaWrite_Per_Sec'),
 ('db_005', 'AWS'),
 ('db_005', 'AIOS'),
 ('db_005', 'DbTime'),
 ('db_005', 'Sess_Active'),
 ('db_005', 'Physical_Read_Per_Sec'),
 ('db_005', 'Login_Per_Sec'),
 ('db_005', 'User_Commit'),
 ('db_005', 'Logic_Read_Per_Sec'),
 ('db_005', 'Redo_Per_Sec'),
 ('db_005', 'Exec_Per_Sec'),
 ('db_005', 'New_Tbs_Free_Gb'),
 ('db_005', 'New_Tbs_Used_Pct'),
 ('db_010', 'tnsping_result_time'),
 ('db_010', 'LFSync_Per_Sec'),
 ('db_010', 'DbTime'),
 ('db_010', 'Physical_Read_Per_Sec'),
 ('db_010', 'L